In [1]:
import hpc
import numpy as np
import matplotlib.pyplot as plt
import pykat as pk
import pykat.optics.gaussian_beams as gb
import pykat.math as pkmath
from PIL import Image
from statistics import mean
import scipy as sp
import math
from scipy import ndimage
from scipy.optimize import least_squares

%matplotlib inline 
%config InlineBackend.figure_format = 'retina'
from pylab import rcParams
rcParams['figure.figsize'] = 10, 5


                                              ..-
    PyKat 1.2.2           _                  '(
                          \`.|\.__...-""""-_." )
       ..+-----.._        /  ' `            .-'
   . '            `:      7/* _/._\    \   (
  (        '::;;+;;:      `-"' =" /,`"" `) /
  L.        \`:::a:f            c_/     n_'
  ..`--...___`.  .    ,
   `^-....____:   +.      www.gwoptics.org/pykat



C:\Users\fritz\anaconda3\lib\site-packages\pykat\__init__.py:65: UserWarning: Could not find the finesse executable 'kat'or you do not have the permissions to run it.
  warn(msg)


In [13]:
#requires at least four images of beat note and image of probe beam
#give decompose function below loaded in numpy arrays for ^ 

def beam_shift_center(image):
    #centers a beam's center of mass at the center of the frame. Note: accomplishes this by rolling array values over the edges of the frame, so for best results, center beam in frame as much as possible.
    cm = ndimage.measurements.center_of_mass(image)
    ishift = int(round(len(image[0])/2-cm[1]))
    jshift = int(round(len(image)/2-cm[0]))
    beampic = image
    beampic = np.roll(beampic,ishift,axis = 1)
    beampic = np.roll(beampic,jshift,axis = 0)

    return(beampic)

def fourpoint(piclist):
    arr1 = np.ravel(np.array(piclist[0],dtype='int'))
    arr2 = np.ravel(np.array(piclist[1],dtype='int')) #converts to numpy arrays for faster operation
    arr3 = np.ravel(np.array(piclist[2],dtype='int'))
    arr4 = np.ravel(np.array(piclist[3],dtype='int'))
    phase = np.empty(len(piclist[0])*len(piclist[0][0]))
    mask = np.ones(len(piclist[0])*len(piclist[0][0]),dtype=bool)
    cuts = np.where(arr1 < 15)
    mask[cuts] = False
    p1 = arr1[mask]
    p2 = arr2[mask]
    p3 = arr3[mask]
    p4 = arr4[mask]
    num = p4 - p2
    den = p1 - p3
    pha = np.arctan2(num,den)
    phase[~mask] = 0
    phase[mask] = pha
    phase = np.reshape(phase,(len(piclist[0]),len(piclist[0][0])))
    return(phase)

def subtract_center_phase(phasemap):
    avcenterphase = []
    for i in range(41):
        for j in range(41):
            avcenterphase.append(phasemap[int(len(phasemap)/2)-40+i][int(len(phasemap[0])/2)-40+j])
    average = sum(avcenterphase)/len(avcenterphase)
    centerphase = np.full((len(phasemap),len(phasemap[0])),average)
    return(phasemap-centerphase)

def curvaturebounds(diffphase):
    #Accepts wavefront image of beam with central phase subtracted out, returns indices that represent bounds of beam curvature through x,y
    lxindex, rxindex, lyindex, ryindex = 0,0,0,0
    midldist = int(len(diffphase[0])/2)
    midrdist = -1*midldist
    for i in range(len(diffphase[0])-1):
        if diffphase[int(len(diffphase)/2)][i+1] - diffphase[int(len(diffphase)/2)][i] <= -1*math.pi and int(len(diffphase[0])/2) - (i+1) < midldist and int(len(diffphase[0])/2) - (i+1) > 0:
            lxindex = i+1
            midldist = int(len(diffphase[0])/2)-(i+1)
        if diffphase[int(len(diffphase)/2)][i+1] - diffphase[int(len(diffphase)/2)][i] >= 1*math.pi and int(len(diffphase[0])/2) - (i) > midrdist and int(len(diffphase[0])/2) - (i) < 0:
            rxindex = i+1
            midrdist = int(len(diffphase[0])/2) - (i)
    mid = int(len(diffphase)/2)
    midldist = int(len(diffphase)/2)
    midrdist = -1*midldist
    for j in range(len(diffphase)-1):
        if diffphase[j+1][mid] - diffphase[j][mid] <= -1*math.pi and int(len(diffphase)/2) - (j+1) < midldist and int(len(diffphase)/2) - (j+1) > 0:
            lyindex = j+1
            midldist = int(len(diffphase)/2) - (j+1)
        if diffphase[j+1][mid] - diffphase[j][mid] >= 1*math.pi and int(len(diffphase)/2) - (j) > midrdist and int(len(diffphase)/2) - (j) < 0:
            ryindex = j+1
            midrdist = int(len(diffphase)/2) - (j)
    return([lxindex, rxindex, lyindex, ryindex])

def phaseparabola(x,Rc,j,h):
    #approximates wavefront curvature by fitting a parabola
    k = 2*np.pi/1064e-9
    phase = (k/(2*Rc))*((x-h)**2)+j
    return phase

def gaussianequation(x,P,w,a):
    #returns power, waist size, horz offset
    intensity = (2*P)/(math.pi*(w**2))*np.exp(-2*(x+a)**2/(w**2))
    return intensity

def radcurvapprox(diffphase):
    #approximates x and y wavefront curvatures and takes the mean to approximate radius of curvature of wavefront
    indexlist = curvaturebounds(diffphase)
    xrange = np.linspace(indexlist[0]*6.9e-6,indexlist[1]*6.9e-6,indexlist[1]-indexlist[0])
    yrange = np.linspace(indexlist[2]*6.9e-6,indexlist[3]*6.9e-6,indexlist[3]-indexlist[2])
    xpopt, pcov = sp.optimize.curve_fit(phaseparabola,xrange,diffphase[int(len(diffphase)/2)][indexlist[0]:indexlist[1]],[-1,3,0])
    ypopt, pcov = sp.optimize.curve_fit(phaseparabola,yrange,diffphase[indexlist[2]:indexlist[3],int(len(diffphase[0])/2)],[-3,0,0])
    meanRC = abs((xpopt[0]+ypopt[0])/2)
    return(meanRC)

def waistsizeapprox(probebeam):
    #measures approx. x and y waist sizes of intensity image of probe beam, returns average to approx beam parameter
    normprob = probebeam/(np.sum(np.sum(probebeam))*((6.9e-6)**2))
    probdata = normprob[int(len(normprob)/2)]
    yprobdata = normprob[:,int(len(normprob[0])/2)]
    x = np.linspace(-6.9e-6*(int(len(probebeam[0])/2)),6.9e-6*(int(len(probebeam[0])/2)),len(probebeam[0])) #dimensions of probe beam image
    y = np.linspace(-6.9e-6*(int(len(probebeam)/2)),6.9e-6*(int(len(probebeam)/2)),len(probebeam))
    x1popt, pcov = sp.optimize.curve_fit(gaussianequation,x,probdata)
    y1popt, pcov = sp.optimize.curve_fit(gaussianequation,y,yprobdata)
    meanw = abs((x1popt[1]+y1popt[1])/2)
    return(meanw)

def HG_mode_content_1(w, rc, beam_data, X, Y, n, m, Print=True):
    #decomposes beam using optimal parameter with values calculated above
    import pykat.optics.gaussian_beams as gb
    q = gb.BeamParam(w=w,rc=rc)
    #print(q.Rc)
    sq_sum = 0
    c_nm = []
    for i in range(0, n+1):
        for j in range(0, m+1):
            HG_mode = gb.HG_mode(q, n=i, m=j)
            HG_field = HG_mode.Unm(X,Y)
            k_nm = np.sum(np.multiply(np.conj(beam_data), HG_field))*np.diff(X)[0]*np.diff(Y)[0]
            sq_sum += np.abs(k_nm)**2
            if Print==True:
                print('%i%i: Mag: %.8E     Ang: %.2F'  % (i, j, np.abs(k_nm), np.angle(k_nm, deg=True)))
                c_nm.append([(i,j), np.abs(k_nm), np.angle(k_nm, deg=True)])
            else:
                c_nm.append([(i,j), np.abs(k_nm), np.angle(k_nm, deg=True)])
    print('Squared sum:', sq_sum)
    return c_nm

In [14]:
def decompose(probebeam,beat1,beat2,beat3,beat4):
    piclist = [beam_shift_center(beat1),beam_shift_center(beat2),beam_shift_center(beat3),beam_shift_center(beat4)]
    diffphase = subtract_center_phase(fourpoint(piclist))
    rc = radcurvapprox(diffphase)
    w = waistsizeapprox(beam_shift_center(probebeam))
    x = np.linspace(-6.9e-6*(int(len(probebeam[0])/2)),6.9e-6*(int(len(probebeam[0])/2)),len(probebeam[0])) #dimensions of probe beam image
    y = np.linspace(-6.9e-6*(int(len(probebeam)/2)),6.9e-6*(int(len(probebeam)/2)),len(probebeam))
    n,m = 5,5 #highest mode order you'd like decomposition to return
    normprob = probebeam/(np.sum(np.sum(probebeam))*((6.9e-6)**2))
    beamphi = np.sqrt(normprob)*np.exp(1j*diffphase) #includes phase info in decomposition
    return(HG_mode_content_1(w,rc,beamphi,x,y,n,m))
    

In [15]:
probebeam = beam_shift_center(np.load("probe_beam_0608_0.npy"))
newprobe = beam_shift_center(probebeam[100:400,200:500])
def beamcrop(image):
    img = beam_shift_center(image)
    return(beam_shift_center(img[100:400,200:500]))
beat1 = beamcrop(np.load("0608_0.npy"))
beat2 = beamcrop(np.load("0608_1.npy"))
beat3 = beamcrop(np.load("0608_2.npy"))
beat4 = beamcrop(np.load("0608_3.npy"))

decompose(newprobe,beat1,beat2,beat3,beat4)


00: Mag: 9.61450912E-01     Ang: 32.46
01: Mag: 1.63668248E-02     Ang: 11.44
02: Mag: 6.28137042E-02     Ang: -115.42
03: Mag: 3.92587014E-03     Ang: -124.92
04: Mag: 3.51937881E-02     Ang: -10.00
05: Mag: 2.06611424E-03     Ang: -15.13
10: Mag: 3.47902885E-02     Ang: -130.88
11: Mag: 1.09739409E-02     Ang: 53.21
12: Mag: 3.62165656E-03     Ang: -166.15
13: Mag: 1.19909998E-02     Ang: -82.83
14: Mag: 1.80987965E-03     Ang: 83.64
15: Mag: 1.02573124E-02     Ang: 157.03
20: Mag: 1.08150262E-01     Ang: -171.91
21: Mag: 2.25098567E-02     Ang: 127.51
22: Mag: 3.22337283E-02     Ang: -104.72
23: Mag: 2.47337125E-03     Ang: 70.89
24: Mag: 3.41315917E-02     Ang: 133.85
25: Mag: 5.20631293E-03     Ang: 58.16
30: Mag: 2.70059303E-02     Ang: 118.26
31: Mag: 1.45392282E-02     Ang: -101.34
32: Mag: 6.86822155E-03     Ang: -46.95
33: Mag: 4.93373434E-03     Ang: 141.66
34: Mag: 3.80657337E-03     Ang: -179.79
35: Mag: 9.82644901E-04     Ang: 14.09
40: Mag: 7.04045792E-02     Ang: 5.79
4

[[(0, 0), 0.9614509116973294, 32.46102647141144],
 [(0, 1), 0.016366824798605743, 11.436731073205237],
 [(0, 2), 0.06281370418258825, -115.41656357649278],
 [(0, 3), 0.003925870136032355, -124.91656186631909],
 [(0, 4), 0.035193788095259554, -10.00454819803922],
 [(0, 5), 0.002066114237497668, -15.130278535047967],
 [(1, 0), 0.034790288495165064, -130.87696898017526],
 [(1, 1), 0.010973940878609933, 53.21485915009802],
 [(1, 2), 0.003621656558352224, -166.14891451437447],
 [(1, 3), 0.011990999808140237, -82.83172624564148],
 [(1, 4), 0.0018098796496297338, 83.63859051516816],
 [(1, 5), 0.010257312354649722, 157.0336519993124],
 [(2, 0), 0.1081502623151682, -171.91043238449996],
 [(2, 1), 0.022509856713096858, 127.51030980482317],
 [(2, 2), 0.03223372830686034, -104.72320560922599],
 [(2, 3), 0.0024733712457907274, 70.8926703726536],
 [(2, 4), 0.03413159173715325, 133.85140491248177],
 [(2, 5), 0.005206312930424328, 58.156794087155525],
 [(3, 0), 0.027005930254216298, 118.25584266679216